## <center style="color: #66d">Projet A - Volcans remarquables</center>

### 1. Origine des données - DBPedia

Cette année, le principe de chacun des projets consiste à récupérer des données sur DBPedia. 

<img src="DBPedia.png" width="120">

DBpedia est un projet universitaire et communautaire d'exploration et extraction automatique de données dérivées de Wikipédia. Son principe est de proposer une version structurée et sous forme de données normalisées au format RDF des contenus encyclopédiques de chaque page de Wikipédia. Il existe plusieurs versions de DBpedia et dans plusieurs langues. Les trois versions principales sont la version anglaise (http://dbpedia.org/sparql), la versions française (http://fr.dbpedia.org) et la version allemande (http://de.dbpedia.org/).

La version qui a été utilisée pour récupérer les données qui vous sont fournies est la version anglaise c’est à dire http://dbpedia.org/ car c’est la plus complète. Cependant il est tout à fait possible d’adapter les différentes requêtes aux autres chapitres multilingues de DBpedia (ex: la version française) en tenant compte des différences entre les chapitres.



### 2. Format des données - RDF

Sur DBPedia, les données sont représentées au format
<a href="https://fr.wikipedia.org/wiki/Resource_Description_Framework">RDF</a>
(Resource Description Framework). RDF est un modèle de graphe destiné à décrire de façon formelle des ressources et leurs métadonnées, de façon à permettre le traitement automatique de telles descriptions. Développé par le <a href="https://www.w3.org/">World Wide Web Consortium</a> (W3C en abrégé), RDF est un des langages de base du Web sémantique.

<img src="Rdf_logo.svg" width="100">

Un document RDF est constitué d'un ensemble de triplets. Un triplet RDF est une association (sujet, prédicat, objet).

* Le sujet représente la ressource à décrire.
* Le prédicat est une propriété de la ressource.
* L’objet donne la valeur de la propriété, et peut correspondre à une donnée numérique ou textuelle, ou à autre ressource.

Les ressources et les prédicats sont représentés à l'aide d'une URL.

Exemple : pour observer l'ensemble des propriétés de la ressource <code>&lt;http://</code><code>dbpedia.org</code><code>/resource/Acatenango&gt;</code> avec leur valeur il suffit d'actionner le lien : http://dbpedia.org/resource/Acatenango

On y apprend ainsi que :<br>
<code>&lt;http://</code><code>dbpedia.org</code><code>/resource/Acatenango&gt;</code> <code>dbo:elevation 3976.000000</code>

Ce qui peut s'exprimer en français par : L'Acatenango s'élève à une altitude (dbpedia ontology : elevation) de 3976 m.

Note: pour représenter les URLs des ressources et des propriétés, DBPedia utilise un certain nombre de préfixes prédéfinis. Ainsi l'URL <code>&lt;dbo:elevation&gt;</code> correspond à <code>&lt;http://</code><code>dbpedia.org</code><code>/ontology/elevation&gt;</code> obtenue en remplaçant le préfixe par sa valeur.


### 3. Récupération de données sur DBPedia - SPARQL

<a href="https://fr.wikipedia.org/wiki/SPARQL">SPARQL</a> est un langage de requête et un protocole qui permet de rechercher, d'ajouter, de modifier ou de supprimer des données RDF disponibles à travers Internet. Son nom est un acronyme récursif qui signifie : "SPARQL Protocol And RDF Query Language".

Voici un exemple simple de requête SPARQL, qui s'interprète comme "Quelle est la ressource dont le sujet principal est l'Acatenango ?", et va nous retourner l'adresse de la page wikipédia consacrée à l'Acatenango :

<code>SELECT ?wiki  WHERE { </code><code>&lt;http://</code><code>dbpedia.org</code><code>/resource/Acatenango&gt;</code><code>  foaf:isPrimaryTopicOf  ?wiki  }</code>

Il est possible de soumettre de telles requêtes sur le point d'accès dédié de DBPedia : http://dbpedia.org/sparql.

<div style="background-color:#eef;padding:10px;border-radius:3px; margin-top: 1.33em">
Soumettez la requête précédente via le point d'accès SPARQL de DBPedia pour observer
la réponse obtenue, et vérifier que l'information retournée correspond bien à l'adresse de la page wikipédia demandée : <code>http://en.wikipedia.org/wiki/Acatenango</code>.
</div>

SPARQL est un langage puissant, qui permet d'émettre des requêtes complexes. Pour plus d'informations sur SPARQL et la façon d'utiliser DBPedia, il ne sera pas inutile de consulter le <a href="http://fr.dbpedia.org/sparqlTuto/tutoSparql.html">tutoriel en ligne</a>.

### 4. Données sur les volcans

Les ressources concernant les volcans disponibles sur DBPedia sont du type <a href="https://dbpedia.org/ontology/Volcano"><code>dbo:Volcano</code></a>. La requête suivante en demande la liste, avec leur nom, l'adresse de la page Wikipédia qui les décrit, leur altitude, latitude, longitude, la date ou l'époque de leur dernière éruption, le texte et l'URL de l'image qui les décrivent sur Wikipédia :

__Remarque importante__

<p>Cela n'est pas nécessaire dans l'immédiat, mais si vous désirez réinitialiser le contenu de votre base de données, il faudra exécuter dans l'ordre, l'ensemble des cellules présentes dans la suite de ce notebook.  Si les données sources ont été modifiées, il faudra peut-être intervenir à la marge sur certaines parties du code de nettoyage des données.</p>

<p>De même, pour compléter et/ou modifier vos données, vous devrez modifier la requête SPARQL présente dans la cellule ci-dessus, et éventuellement nettoyer les nouvelles données obtenues en complétant le notebook avec le code python nécessaire.
</p>

<p>Toutefois, avant de vous aventurer à modifier la requête SPARQL, il sera pertinent de tester votre nouvelle requête via le point d'entrée interactif de DBPedia, en ajoutant une clause LIMIT(10) par exemple, pour éviter de surcharger le serveur, et d'être obligé d'attendre les résultats trop longtemps.</p> 

__4.1 Enregistrement du notebook et récupération de son nom dans la variable notebook_name__

In [ ]:
%%javascript

// Enregistrement des éventuelles modifications de la requête SPARQL
IPython.notebook.save_notebook()

// Enregistrement du nom du présent notebook dans la variable python notebook_name
var kernel = IPython.notebook.kernel;
var thename = window.document.getElementById("notebook_name").innerHTML;
var command = "notebook_name = " + "'"+thename+"'";
kernel.execute(command);
element.text(command)

<IPython.core.display.Javascript object>

__4.2 Définition des fonctions utilisées par la suite__

In [5]:
#
# Emission d'un requête SPARQL vers le point d'entrée DBPedia
# et récupération du résultat dans un fichier csv
#
# id : metadata.id de la cellule avec la requête SPARQL, et nom du fichier csv
#
def dbpedia_sparql_to_csv(cell_id):

    query = get_cell_by_id(cell_id)['source']
    url = display_dbpedia_links(query)['csv']
    http_request_to_file(url,'{}.csv'.format(cell_id))

#
# Récupère une cellule du présent notebook
#
def get_cell_by_id(cell_id):

    # https://discourse.jupyter.org/t/extract-specific-cells-from-students-notebooks/7951/4
    import os
    import nbformat as nbf
    filename = "{}.ipynb".format(notebook_name)
    notebook = nbf.read(filename, nbf.NO_CONVERT)
    return [c for c in notebook.cells if 'id' in c['metadata'] and c['metadata']['id'] == cell_id][0]

#
# Renvoie l'url d'une requête vers le point d'entré SPARQL de DBPedia
#
def dbpedia_sparql_url(query,fmt):

    # https://stackoverflow.com/questions/40557606/how-to-url-encode-in-python-3
    from urllib.parse import urlencode, quote_plus
    url = "https://dbpedia.org/sparql"
    params = {
        "default-graph-uri" : "http://dbpedia.org",
        "query" : query,
        "format" : fmt,
        "timeout" : 30000,
        "signal_void" : "on",
        "signal_unconnected" : "on"
    }
    return "{}?{}".format(url,urlencode(params,quote_via=quote_plus))

#
# Affiche et renvoie les liens pour une requête SPARQL sur le point d'entrée DBPedia
# avec un résultat au format html, json, ou csv
#
def display_dbpedia_links(query):
    
    html_url = dbpedia_sparql_url(query,'text/html')
    json_url = dbpedia_sparql_url(query,'application/sparql-results+json')
    csv_url = dbpedia_sparql_url(query,'text/csv')
    
    # https://stackoverflow.com/questions/48248987/inject-execute-js-code-to-ipython-notebook-and-forbid-its-further-execution-on-p
    from IPython.display import display, HTML

    html_link = '<a href="{}">HTML</a>'.format(html_url)
    json_link = '<a href="{}">JSON</a>'.format(json_url)
    csv_link = '<a href="{}">CSV</a>'.format(csv_url)

    display(HTML('Requêtes : {}&nbsp;&nbsp;{}&nbsp;&nbsp;{}'.format(html_link,json_link,csv_link)))

    return { "html": html_url, "json": json_url, "csv": csv_url}

#
# Emet une requête http et enregistre le résultat dans un fichier
#
def http_request_to_file(url,filename):
    
    # https://stackoverflow.com/questions/645312/what-is-the-quickest-way-to-http-get-in-python
    import urllib.request
    contents = urllib.request.urlopen(url).read()

    with open(filename,'wb') as f:
        f.write(contents)

#
# Vérifie si une chaîne peut être convertie en float
#
def isfloat(value):
  try:
    float(value)
    return True
  except ValueError:
    return False


__4.3 Récupération des données brutes provenant de DBPedia__

Cette cellule envoie la requête SPARQL au serveur DBPedia, et enregistre le résultat dans le fichier sparql.csv.

In [6]:
raw_filename = 'sparql'
dbpedia_sparql_to_csv(raw_filename)

__4.4 Nettoyage des données__

La cellule suivante relit le fichier des données brutes dans le dictionnaire nommé <code>volcans</code>, puis les cellules consécutives modifient ces données en mémoire pour les nettoyer.

In [7]:
#
# Lecture du fichier d'origine, avec suppression des vrais-faux doublons
#
import csv

volcans = {}
with open('{}.csv'.format(raw_filename),encoding="utf-8") as csvfile:
    reader = csv.DictReader(csvfile,delimiter=',')
    for row in reader:
        if not row['name'] in volcans:
            volcans[row['name']] = row

In [8]:
#
# Fix absence de nom
#
if '' in volcans:
    del volcans['']

In [9]:
#
# Chasse aux doublons non homonymes
#
to_be_deleted = []
for v in volcans:
    wiki = volcans[v]['volcano']
    found = [k for k in volcans if volcans[k]['volcano'] == wiki]
    if len(found) > 1 and not found[1] in to_be_deleted:
        to_be_deleted.append(found[1])

for k in to_be_deleted:
    del volcans[k]
    print('deleted {}'.format(k))

deleted Volcano in Chile
deleted Parinacota


In [10]:
#
# Suppression des listes, en général non pertinentes
#
import re

for v in volcans:
    text = volcans[v]['abstract']
    if '*' in text:
        text = re.sub(r'(?smu)\*.*$', '', text)
        text = re.sub(r'\s*:\s*$', '.', text)
        volcans[v]['abstract'] = text

__4.5 Mise en forme des dates d'éruption__

In [11]:
#
# Mise en forme et traduction des dates d'éruption
#
from datetime import date
now = date.today().year

periods = {
    'Chibanian': ('au Chibanien (Pléistocène moyen)', -781000),
    'Holocene': ("durant l'Holocène", -12000),
    'Paleocene': ('au Paléocène', -66000000),
    'Pleistocene': ('au Pléistocène', -2580000),
    'Middle_Pleistocene': ('au Pléistocène', -2580000),
}
months = {
    'january': 'janvier',
    'february': 'février',
    'march': 'mars',
    'april': 'avril',
    'may': 'mai',
    'june': 'juin',
    'july': 'juillet',
    'august': 'août',
    'september': 'septembre',
    'october': 'octobre',
    'november': 'novembre',
    'december': 'décembre'
}
special = {
    'Aguilera': ('il y a environ 3000 ans', -3000 + now),
    'Mount Fuppushi': ('il y a 10000 ans', -10000 + now),
    'Mount Kenya': ("il y a 3.1 à 2.6 millions d'années", -3100000 + now),
    'Mount Sinabung': ('2021 (en cours)', now),
    'Mount Amagi': ("il y a 0.2 millions d'années", -200000 + now),
    'Aucanquilcha': ('Pléistocène', -2580000),
    'Cerro Macá': ('1560', 1560),
    'Coropuna': ('il y a environ 950 ans', -950 + now),
    'Mount Kamui': ('1080', 1080),
    'Pomerape': ('il y a environ 106000 ans', -106000 + now),
    'The Cheviot': ("il y a 393 millions d'années", -393000000 + now),
}

for v in volcans:
    year = volcans[v]['year'] if 'year' in volcans[v] else None 
    date = volcans[v]['eruption']
    name = volcans[v]['name']
    
    # Unknown
    if date.lower() == 'unknown' or date.lower() == "'unknown'":
        date = "date inconnue"
        year = -1000000000
    
    # Ongoing
    elif date.lower() == 'ongoing':
        date = "en cours"
        year = now
    
    # Not in historic time
    elif 'historic' in date:
        date = "aucune pendant les temps historiques"
        year = -1000000000
        
    # http://dbpedia.org/resource/Pleistocene
    elif date.startswith('http:') or date in periods:
        date, year = periods[date.split('/').pop()]
    
    # Holocene?
    elif '?' in date:
        period = periods[[k for k in periods.keys() if k in date][0]]
        date = "probablement {}".format(period[0])
        year = period[1]
    
    # possibly ...
    elif 'possibly' in date.lower():
        found = [k for k in periods.keys() if k in date]
        
        # Possibly Holocene
        if len(found):
            period = periods[found[0]]
            date = "peut-être {}".format(period[0])
            year = period[1]
        
        # Possibly 1251
        else:
            year = date.split().pop()
            date = "peut-être en {}".format(year)
            year = int(year)
    
    # Late Pleistocene
    elif 'late' in date.lower():
        found = [k for k in periods.keys() if k in date]
        period = periods[found[0]]
        date = "{} tardif".format(period[0])
        year = period[1]
    
    # Pleistocene time
    elif len([k for k in periods.keys() if k in date]):
        date, year = periods[[k for k in periods.keys() if k in date][0]]
    
    # November to December 1972
    elif len([k for k in months.keys() if k in date.lower()]):
        found = [k for k in months.keys() if k in date.lower()]
        for m in found:
            date = date.lower().replace(m,months[m])
        date = date.replace(' to ',' à ')
        year = int(re.search(r'([0-9]+)',date).group(1))

    # ca. 0.3-0.25 million years ago
    elif 'ca.' in date:
        date = "il y a environ {} millions d'années".format(date.split()[1].replace('-',' à '))
        year = -300000 + now
    
    elif name in special:
        date, year = special[name]
       
    # -350-01-01
    elif year and year.split('-')[0] == '' :
        year = year.split('-')[1]
        date = 'vers {} AEC'.format(year)
        year = -int(year)
        
    # 2460-01-01
    elif year and float(year.split('-')[0]) > 2021:
        year = year.split('-')[0]
        date = 'il y a environ {} ans'.format(year)
        year = -int(year) + now
        
    # 1707-01-01
    elif year and float(year.split('-')[0]) > 1000:
        date = year.split('-')[0]
        year = int(date)

    # 5000
    elif isfloat(date) and float(date) > 2022 and float(date) < 10000 :
        year = date
        date = 'vers {} AEC'.format(date)
        year = -int(year)
        
    # 0560-01-01
    elif isfloat(date) and round(abs(float(date)) / 365.25 / 24 / 3600) < 1500:
        year = int(year.split('-')[0])
        date = "vers l'an {} environ".format(year)
        year = int(year)
    
    # 6.31152E12
    elif isfloat(date):
        idate = round(abs(float(date)) / 365.25 / 24 / 3600)
        date = "il y a environ {} ans".format(idate)
        year = -idate + now
            
    # On ne devrait pas passer par ici
    else:
        print((year if year else '        ')+'\t'+date+'\t'+name+'\t'+volcans[v]['wiki'])
        pass
    
    # Mise à jour des dates dans le dictionnaire
    volcans[v]['eruption_date'] = date
    volcans[v]['eruption_year'] = year

__4.6 Ecriture du fichier des données nettoyées__

In [12]:
#           
# Ecriture du fichier csv à importer dans la base de données
#
fieldnames = list(volcans['Mount Etna'].keys())
fieldnames.remove('year')
fieldnames.remove('eruption')

with open('volcans.csv', 'w', encoding='utf-8', newline='\n') as f:
    writer = csv.DictWriter(f, fieldnames=fieldnames, delimiter=';')
    writer.writeheader()
    for v in volcans:
        writer.writerow({f: volcans[v][f] for f in fieldnames})

__4.7 Création / mise à jour de la base de données__

In [13]:
# relecture du fichier de données
volcans = {}

with open('volcans.csv',encoding="utf-8") as csvfile:
    reader = csv.DictReader(csvfile,delimiter=';')
    for row in reader:
        volcans[row['name']] = row

fieldnames = list(volcans['Mount Etna'].keys())

#
# Mise à jour de la base de données
#
import sqlite3

volcano_dbname = 'volcans.db'
conn = sqlite3.connect(volcano_dbname)
c = conn.cursor()

c.execute("DROP TABLE IF EXISTS volcans")
conn.commit()

c.execute('''CREATE TABLE "volcans" (
    `volcano` TEXT PRIMARY KEY,
    `name` TEXT,
    `wiki` TEXT,
    `elevation` INTEGER,
    `lat` REAL,
    `lon` REAL,
    `eruption_date` TEXT,
    `eruption_year` INTEGER,
    `abstract` TEXT,
    `photo` TEXT
)''')
conn.commit()

request = 'INSERT INTO volcans ({}) VALUES ({})'.format(','.join(fieldnames),','.join(['?']*len(fieldnames)))
for v in volcans:
    c.execute(request,[volcans[v][k] for k in fieldnames])
conn.commit()